In [126]:
import numpy as np
import math
import collections
import random
from scipy.optimize import linprog
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [127]:
def hist(samp_list, min_s, max_s):
    h = [0] * (max_s - min_s + 1)
    for i in samp_list:
        h[i-min_s] += 1
    return h

def makeFingerPrint(samp_list):
    max_s = max(samp_list)
    min_s = min(samp_list)
    h1 = hist(samp_list, min_s, max_s)
    f = hist(h1, 0, max(h1))
    f = f[1:]
    return f

In [128]:
#FOR UNSEEN
def poisspdf(x, l):
    v1 = math.pow(l, x)
    v2 = math.factorial(x)
    v3 = math.pow(math.e, -l)
    final = (v1 / v2) * v3
    return final

def getSampleSize(f):
    k = 0
    for i,entry in enumerate(f):
        k = k + entry * (i+1)
    return k

#check
def getFirstNonZeroIndex(f):
    for i,entry in enumerate(f):
        if(entry != 0):
            return i+1

#check
def getLastNonZeroIndex(f):
    for i, entry in reversed(list(enumerate(f))):
        if(entry != 0):
            return i+1
        return -1

def getProbabilityMass(x, histx):
    pmass = 0
    for i, x_entry in enumerate(x):
        pmass = pmass + x_entry * histx[i]
    return pmass

def init_XLP(xLPmax, xLPmin, gridFactor):
    max_pow = math.ceil(math.log(xLPmax/xLPmin) / math.log(gridFactor))
    xLP = []
    for i in range(0, max_pow+1):
        xLP.append(xLPmin * math.pow(gridFactor, i))
    return xLP

def init_objf(szLPx, szLPf, fLP):
    objf = [0] * (szLPx + 2*szLPf)
    j = 0
    for i in range(szLPx, len(objf), 2):
        objf[i] = 1 / math.sqrt(fLP[j] + 1)
        j = j + 1
    j = 0
    for i in range(szLPx+1, len(objf), 2):
        objf[i] = 1 / math.sqrt(fLP[j] + 1)
        j = j + 1
    return objf

def init_A_b(szLPf, szLPx, xLP, fLP, k):
    #A = [[0] * (szLPx + 2 * szLPf)] * 2*szLPf
    A = []
    for i in range(2*szLPf):
        tmp = []
        for j in range(szLPx + 2 * szLPf):
            tmp.append(0)
        A.append(tmp)
    
    b = [0] * (2*szLPf)
    for i in range (0, szLPf):
        for j in range(0, szLPx):
            A[2 * i][j] = poisspdf(i+1, k * xLP[j])
        for k in range(0, szLPx):
            A[2*i+1][k] = -1 * A[2*i][k]
        A[2 * i][szLPx + 2 * i] = -1
        A[(2 * i)  + 1][szLPx + (2 * i)  + 1] = -1
        b[2 * i] = fLP[i]
        b[(2 * i)  + 1] = -fLP[i]
    
    return A,b

def rescale_cond(A, Aeq, xLP, szLPx):
    for i in range(0, szLPx):
        for j in range(0, len(A)):
            A[j][i] = A[j][i]/xLP[i]
        Aeq[i] = Aeq[i]/xLP[i]
    return A, Aeq

    
def unseen(f):
    k = getSampleSize(f)
    
    gridFactor = 1.05
    alpha = 0.5
    
    xLPmin = 1 / (k * max(10,k))
    print(xLPmin)
    
    min_i = getFirstNonZeroIndex(f) #Returning index + 1
    print("min_i", min_i)
    
    if min_i > 1:
        xLPmin = min_i/k
    print("xLPmin", xLPmin)
    
    maxLPIters = 1000
    x = [0]
    histx = [0]
    
    fLP = [0] * len(f)
    for i in range(0, len(f)):
        i_m = i+1
        if(f[i] > 0):
            wind = [max(1, i_m - math.ceil(math.sqrt(i_m))), 
                    min(i_m + math.ceil(math.sqrt(i_m)), len(f))]
            
            low_ind = wind[0] - 1
            high_ind = wind[1]
            sum_f = sum(f[low_ind : high_ind])
            if( sum_f < math.sqrt(i_m)):
                x.append(i_m/k)
                histx.append(f[i])
                fLP[i] = 0
            else:
                fLP[i] = f[i]
    print("fLP", fLP)
    print("histx", histx)
    print("x", x)
    
    #If no LP portion, retun the empirical histogram
    fmax = getLastNonZeroIndex(f)
    if(fmax == -1):
        x.pop(0)
        histx.pop(0)
        return [x, histx]
    
    #Setting up first LP
    LPmass = 1 - getProbabilityMass(x, histx)
    print("LPMASS", LPmass)
    
    print("FMAX----->", fmax)
    z_flp = [0] * math.ceil(math.sqrt(fmax))
    fLP.extend(z_flp)
    print("New FLP", fLP)
    
    szLPf = len(fLP)
    print("szLFf",szLPf)
    
    xLPmax = fmax/k
    print("xLPmax", xLPmax)
    
    xLP = init_XLP(xLPmax, xLPmin, gridFactor)
    
    szLPx = len(xLP)
    print("szLPx", szLPx)
    
    objf = init_objf(szLPx, szLPf, fLP)
    
    A, b = init_A_b(szLPf, szLPx, xLP, fLP, k)
    print(b)
    
    Aeq = [0] * (szLPx+2*szLPf)
    for i in range(0,szLPx):
        Aeq[i] = xLP[i]
    beq = LPmass
    
    A, Aeq = rescale_cond(A, Aeq, xLP, szLPx)
    #print("Aeq",Aeq)
    
    options = {'maxiter':1000, 'disp':False}
    lb = [0] * (szLPx+2*szLPf)
    ub = [float('Inf')] * (szLPx+2*szLPf)
    #print(len(ub))
    #print(len(A),len(A[0]))
    
    
    
#     print("==========================info start===========================");
#     print("objf");
#     print(objf);
#     print("A");
#     print(A);
#     print("b");
#     print(b);
#     print("Aeq");
#     print(Aeq);
#     print("beq");
#     print(beq);
#     print("szLPx");
#     print(szLPx);
#     print("szLPf");
#     print(szLPf);
#     print("options");
#     print(options);
#     print("===========================info end============================");

    
    
    print("objf", len(objf))
    res = linprog(objf, A, b, [Aeq], beq, list(zip(lb, ub)), options={'disp':False}, method='interior-point')
    print("res.fun", res.fun)
    
    #Second LP
    objf2 = [0] * len(objf)
    print("Len of objf2",len(objf2))
    for i in range(0, szLPx):
        objf2[i] = 1
    
    A2 = []
    for row in A:
        A2.append(row)
    A2.append(objf)
    
    print("A2", len(A2), len(A2[0]))
    
    b.append(res['fun'])
    b2 = b
    print('b2', b2)
    for i in range(0, szLPx):
        objf2[i] = objf2[i]/xLP[i]
    
    res = linprog(objf2, A2, b2, [Aeq], beq, list(zip(lb, ub)), options={'disp':False}, method='interior-point')
    print("---------------------------------------------------------------")
    print("linprog2",res)
    
    sol2 = res['x']
    #print (sol2)
    
    for i in range (0, szLPx):
        sol2[i] = sol2[i] / xLP[i]
    
    print("--->",len(sol2), szLPx)
    print(x)
    #print(xLP)
    
    
    for i in range (0, len(xLP)):
        x.append(xLP[i])
    for i in range (0, len(sol2)):
        histx.append(sol2[i])
    
    print("=============================")
    print(x)
    print("=============================")
    print(histx)
    print("=============================")
    
    
    ind = np.argsort(x)
    #print(x)
    x.sort()
    
    tempHistX = []
    for i in range (0, len(ind)):
        tempHistX.append(histx[ind[i]])
    histx = tempHistX
    
    print("=============================")
    print(histx)
    print("=============================")
    
    ind = []
    for i in range (0, len(histx)):
        if histx[i] > 0:
            ind.append(i)
            
    tempX = []
    #print(x)
    #check
    print("=====xx========================")
    print(x)
    print("=====xx========================")
    if x is not None:
        for i in range (0, len(ind)):
            tempX.append(x[ind[i]])

        x = tempX
    
    
    tempHistX = []
    for i in range (0, len(ind)):
        tempHistX.append(histx[ind[i]])
    histx = tempHistX
    
    return histx, x

In [129]:
def entropy_estC(f):
    k = getSampleSize(f)
    
    gridFactor = 1.05
    alpha = 0.5
    
    xLPmin = 1 / (k * max(10,k))
    print(xLPmin)
    
    min_i = getFirstNonZeroIndex(f) #Returning index + 1
    print("min_i", min_i)
    
    if min_i > 1:
        xLPmin = min_i/k
    print("xLPmin", xLPmin)
    
    maxLPIters = 1000
    x = [0]
    histx = [0]
    
    fLP = [0] * len(f)
    for i in range(0, len(f)):
        i_m = i+1
        if(f[i] > 0):
            wind = [max(1, i_m - math.ceil(math.sqrt(i_m))), 
                    min(i_m + math.ceil(math.sqrt(i_m)), len(f))]
            
            
            low_ind = wind[0] - 1
            high_ind = wind[1]
            sum_f = sum(f[low_ind : high_ind])
            if( sum_f < math.sqrt(i_m)):
                x.append(i_m/k)
                histx.append(f[i])
                fLP[i] = 0
            else:
                fLP[i] = f[i]
    print("fLP", fLP)
    print("histx", histx)
    print("x", x)
    
    #If no LP portion, retun the empirical histogram and corrected empirical entropy
    fmax = getLastNonZeroIndex(f)
    if(fmax == -1):
        x.pop(0)
        histx.pop(0)
        sumHistX = 0
        for j in range (0, len(histx)):
            sumHistX += histx[j]
        logX = 0
        for j in range (0, len(x)):
            logxX += log(x[j]) * x[j] * histx[j]
        ent = -logX + sumHistX / (2 * k)
        return ent
    
    #Setting up first LP
    LPmass = 1 - getProbabilityMass(x, histx)
    print("LPMASS", LPmass)
    
    print("fmax------>", fmax)
    z_flp = [0] * math.ceil(math.sqrt(fmax))
    fLP.extend(z_flp)
    print("New FLP", fLP)
    
    szLPf = len(fLP)
    print("szLFf",szLPf)
    
    xLPmax = fmax/k
    print("xLPmax", xLPmax)
    
    xLP = init_XLP(xLPmax, xLPmin, gridFactor)
    
    szLPx = len(xLP)
    print("szLPx", szLPx)
    
    objf = init_objf(szLPx, szLPf, fLP)
    
    A, b = init_A_b(szLPf, szLPx, xLP, fLP, k)
    print(b)
    
    Aeq = [0] * (szLPx+2*szLPf)
    for i in range(0,szLPx):
        Aeq[i] = xLP[i]
    beq = LPmass
    
    A, Aeq = rescale_cond(A, Aeq, xLP, szLPx)
    #print("Aeq",Aeq)
    
    options = {'maxiter':1000, 'disp':False}
    lb = [0] * (szLPx+2*szLPf)
    ub = [float('Inf')] * (szLPx+2*szLPf)
    #print(len(ub))
    #print(len(A),len(A[0]))
    
    
    
#     print("==========================info start===========================");
#     print("objf");
#     print(objf);
#     print("A");
#     print(A);
#     print("b");
#     print(b);
#     print("Aeq");
#     print(Aeq);
#     print("beq");
#     print(beq);
#     print("szLPx");
#     print(szLPx);
#     print("szLPf");
#     print(szLPf);
#     print("options");
#     print(options);
#     print("===========================info end============================");

    
    
    print("objf", len(objf))
    res = linprog(objf, A, b, [Aeq], beq, list(zip(lb, ub)), options={'disp':False}, method='interior-point')
    print("value of fun")
    print(res.fun)
    sol = res['x']
    
    #Second LP
    if min_i < 2:
        objf2 = [0] * len(objf)
        for i in range(0, szLPx):
            objf2[i] = 1

        A2 = []
        for row in A:
            A2.append(row)
        A2.append(objf)

        b.append(res['fun'])
        b2 = b
        for i in range(0, szLPx):
            objf2[i] = objf2[i]/xLP[i]

        res = linprog(objf2, A2, b2, [Aeq], beq, list(zip(lb, ub)), options={'disp':False}, method='interior-point')
        print("---------------------------------------------------------------")
        print(res.fun)
    
        sol2 = res['x']
        print (sol2)
    else:
        sol2 = sol
    
    for i in range (0, szLPx):
        sol2[i] = sol2[i] / xLP[i]
    
    print(sol2)
    print(x)
    print(xLP)
    
    # append LP solution to empirical portion of histogram

    firstNonZeroIndex = -1
    # find first non-zero index
    for i in range(0, len(f)):
        if(f[i] > 0):
            firstNonZeroIndex = i
            break
    print(firstNonZeroIndex)
    ent = 0
    if firstNonZeroIndex == -1:
        # check
        for j in range (0, szLPx):
            ent += sol2[j] * -1 * xlp[j] * math.log(xLP[j])
    else:
        positiveXIndices = []
        for j in range (0, len(x)):
            if x[j] > 0:
                positiveXIndices.append(j)
                
        histXPositiveSum = 0
        histXPositive = []
        for j in range (0, len(positiveXIndices)):
            histXPositive.append(histx[positiveXIndices[j]])
            histXPositiveSum += histx[positiveXIndices[j]]
        
        xPositive = []
        for j in range (0, len(positiveXIndices)):
            xPositive.append(x[positiveXIndices[j]])
            
        histXLogXProduct = 0
        for j in range (0, len(histXPositive)):
            histXLogXProduct -= histXPositive[j] * xPositive[j] * math.log(xPositive[j])
            
        sol2XLPProcut = 0
        for j in range (0, szLPx):
            sol2XLPProcut += sol2[j] * xLP[j] * math.log(xLP[j])
        
        ent = histXLogXProduct + histXPositiveSum / (2 * k) - sol2XLPProcut
    
    
    for i in range (0, len(xLP)):
        x.append(xLP[i])
    for i in range (0, len(sol2)):
        histx.append(sol2[i])
    
    ind = np.argsort(x)
    print (x)
    x.sort()
    
    print(ind)
    print(x)
    
    tempHistX = []
    for i in range (0, len(ind)):
        tempHistX.append(histx[ind[i]])
    histx = tempHistX
    
    ind = []
    for i in range (0, len(histx)):
        if histx[i] > 0:
            ind.append(i)
            
    #check
    if x is not None:
        tempX = []
        for i in range (0, len(ind)):
            tempX.append(x[ind[i]])

        x = tempX
    
    
    tempHistX = []
    for i in range (0, len(ind)):
        tempHistX.append(histx[ind[i]])
    histx = tempHistX
    
    return ent

In [138]:
n = 10000
k = 57
f = [5,10,2,5,1]
h, x = unseen(f)
print("True Entropy = ")
#output entropy of the true distribution, Unif[n]
trueEntropy = math.log(n)
print(trueEntropy)

print("Empirical Entropy = ")
#output entropy of the empirical distribution of the sample
empiricalEntropy = 0
for i in range (0, len(f)):
    empiricalEntropy -= f[i] * (i + 1) / k * math.log((i + 1) / k)
print(empiricalEntropy)

print("Estimated Entropy = ")
#output entropy of the recovered histogram, [h,x]
estimatedEntropy = 0
if x is not None:
    for i in range(0, len(x)):
        estimatedEntropy -= h[i] * x[i] * math.log(x[i])
print(estimatedEntropy)

e2 = entropy_estC(f)
print("====================================================================")

print(trueEntropy)
print(empiricalEntropy)
print(estimatedEntropy)
print(e2)

0.00031887755102040814
min_i 1
xLPmin 0.00031887755102040814
fLP [5, 10, 2, 5, 1]
histx [0]
x [0]
LPMASS 1
FMAX-----> 5
New FLP [5, 10, 2, 5, 1, 0, 0, 0]
szLFf 8
xLPmax 0.08928571428571429
szLPx 117
[5, -5, 10, -10, 2, -2, 5, -5, 1, -1, 0, 0, 0, 0, 0, 0]
objf 133
res.fun 5.966517221120014
Len of objf2 133
A2 17 133
b2 [5, -5, 10, -10, 2, -2, 5, -5, 1, -1, 0, 0, 0, 0, 0, 0, 5.966517221120014]
---------------------------------------------------------------
linprog2      con: array([-2.24329106e-08])
     fun: 21.055302623023447
 message: 'Optimization terminated successfully.'
     nit: 18
   slack: array([ 5.84811666e-08, -5.84289399e-08,  6.56988259e+00,  1.08042647e-07,
        1.96533213e-07,  7.64056898e-01,  3.26665140e+00,  1.03408740e-07,
       -2.12492037e-08,  2.12524167e-08, -6.34092336e-08,  7.00207966e-01,
       -6.77807053e-08,  6.96222597e-01, -8.58934538e-08,  8.14459313e-01,
       -6.74848764e-08])
  status: 0
 success: True
       x: array([3.32944137e-11, 3.18901252

/anaconda3/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:1262: OptimizeWarning:

Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.

/anaconda3/lib/python3.6/site-packages/scipy/optimize/_linprog_ip.py:1274: OptimizeWarning:

Solving system with option 'sym_pos':False failed. This may happen occasionally, especially as the solution is approached. However, if you see this frequently, your problem may be numerically challenging. If you cannot improve the formulation, consider setting 'lstsq' to True. Consider also setting `presolve` to True, if it is not already.

